In [1]:
VERSION

v"1.8.1"

In [2]:
ENV["COLUMNS"] = 1000
ENV["LINES"] = 20;

In [3]:
# using Pkg
# Pkg.add("BenchmarkTools")
# Pkg.update("BenchmarkTools")
# Pkg.build("BenchmarkTools")

In [5]:
using DataFrames
using GLMakie
using JLD2
using BenchmarkTools

using Revise
using Astroshaper
import SPICE

In [6]:
meta_kernel = "/Users/masanorikanamaru/Dropbox/spice/hera/kernels/mk/hera_study_PO_EMA_2024_shapeViewer.tm"

SPICE.furnsh(meta_kernel)

In [7]:
et_start = SPICE.utc2et("2027-02-09T22:00:00")
et_end   = SPICE.utc2et("2027-02-11T22:00:00")
step     = 3600/20

et_range = et_start:step:et_end

@show et_range
@show length(et_range);

et_range = 8.554824691849957e8:180.0:8.556552691849957e8
length(et_range) = 961


In [8]:
# save_range = findall(et_range .> et_range[end] - 7.63262 * 3600)
save_range = 1:length(et_range)

@show save_range[begin]
@show save_range[end]
@show length(save_range);

save_range[begin] = 1
save_range[end] = 961
length(save_range) = 961


In [9]:
sun_d1 = spkpos("SUN",       et_range, "DIDYMOS_FIXED",   "None", "DIDYMOS")
sun_d2 = spkpos("SUN",       et_range, "DIMORPHOS_FIXED", "None", "DIMORPHOS")
d1_d2  = spkpos("DIDYMOS",   et_range, "DIMORPHOS_FIXED", "None", "DIMORPHOS")
d2_d1  = spkpos("DIMORPHOS", et_range, "DIDYMOS_FIXED",   "None", "DIDYMOS");

In [10]:
D1_TO_D2 = pxform("DIDYMOS_FIXED",   "DIMORPHOS_FIXED", et_range)
D2_TO_D1 = pxform("DIMORPHOS_FIXED", "DIDYMOS_FIXED",   et_range)

D1_TO_J2000 = pxform("DIDYMOS_FIXED",   "J2000", et_range)
D2_TO_J2000 = pxform("DIMORPHOS_FIXED", "J2000", et_range);

In [11]:
SPICE.kclear()

In [12]:
fig = Figure()
ax = Axis3(fig[1, 1], aspect=:data)

# scatter!(0,0,0)
# scatter!(sun_d1.x, sun_d1.y, sun_d1.z, color=:orange)
# scatter!(d2_d1.x, d2_d1.y, d2_d1.z, color=:skyblue)
# scatter!(sun_d1.x, sun_d1.y, sun_d1.z, color=:orange)
# scatter!(d2_d1.x, d2_d1.y, d2_d1.z, color=:skyblue)

scatter!([r[1] for r in sun_d1], [r[2] for r in sun_d1], [r[3] for r in sun_d1], color=:orange, size=2)

display(fig)

GLMakie.Screen(...)

# Shape Models: Didymos & Dimorphos

In [15]:
shapedir = "/Users/masanorikanamaru/Dropbox/spice/hera/kernels/dsk"

shapepath1 = joinpath(shapedir, "g_50677mm_rad_obj_dida_0000n00000_v001.obj")
shapepath2 = joinpath(shapedir, "g_06650mm_rad_obj_didb_0000n00000_v001.obj")

shape1 = ShapeModel(shapepath1; scale=1000, find_visible_facets=true, save_shape=false)
shape2 = ShapeModel(shapepath2; scale=1000, find_visible_facets=true, save_shape=false);

In [16]:
println(shape1)
println(shape2)

Shape model
-----------
Nodes             : 1000
Faces             : 1996
Surface area      : 1.9587914739775863e6
Volume            : 2.485481753075182e8
Equivalent radius : 390.03836143582186
Center-of-Figure  : [2.484674354494825e-6, -2.0997384259416257e-6, 2.190089591373722e-6]
Inertia tensor    : 
    | Ixx Ixy Ixz |   [0.0, 0.0, 0.0]
    | Iyx Iyy Iyz | = [0.0, 0.0, 0.0]
    | Izx Izy Izz |   [0.0, 0.0, 0.0]

Shape model
-----------
Nodes             : 1538
Faces             : 3072
Surface area      : 53920.63065752515
Volume            : 1.1314114665400542e6
Equivalent radius : 64.64138537338498
Center-of-Figure  : [2.009646200056278e-16, 3.5852088209004e-16, -2.9131831316015805e-15]
Inertia tensor    : 
    | Ixx Ixy Ixz |   [0.0, 0.0, 0.0]
    | Iyx Iyy Iyz | = [0.0, 0.0, 0.0]
    | Izx Izy Izz |   [0.0, 0.0, 0.0]



# TPM only on Didymos

In [20]:
thermo_params = ThermoParams(  # [Michel+2016; Naidu+2020]
    A_B   = 0.059,  # Bolometric  Bond albedo
    A_TH  = 0.0,
    k     = 0.125,
    ρ     = 2170.,
    Cp    = 600.,
    ϵ     = 0.9,
    t_bgn = et_range[begin],
    t_end = et_range[end],
    Nt    = length(et_range),
    z_max = 0.6,
    Nz    = 41,
    P     = SPICE.convrt(DIDYMOS[:P], "hours", "seconds"),
)

Thermophysical parameters
-------------------------
A_B   : 0.059
A_TH  : 0.0
k     : 0.125
ρ     : 2170.0
Cp    : 600.0
ϵ     : 0.9
-------------------------
t_bgn : 8.554824691849957e8
t_bgn : 105180.37410616312 (Normalized by period P)
t_end : 8.556552691849957e8
t_end : 105201.61962468656 (Normalized by period P)
Nt    : 961
Δt    : 180.0000000000556
Δt    : 0.022130748461919817 (Normalized by period P)
-------------------------
z_max : 0.6
z_max : 6.057014348123621 (Normalized by skin depth l)
Nz    : 41
Δz    : 0.015
Δz    : 0.15142535870309054 (Normalized by skin depth l)
-------------------------
P     : 8133.4800000000005
l     : 0.09905870541414048
Γ     : 403.4228550788862
λ     : 0.07680491551461667
-------------------------


In [99]:
savepath = "tmp.jld2"
run_TPM!(shape1, et_range, sun_d1, thermo_params, savepath, save_range)

jldopen(savepath, "r+") do file
    file["D1_TO_J2000"] = D1_TO_J2000[save_range]
end;

In [95]:
draw(shape1; data=:temperature, colormap=:vik)

GLMakie.Screen(...)

In [96]:
data = load("tmp.jld2");

In [98]:
# data["shape"]
# data["et_range"]
# data["sun"]
# data["thermo_params"]
# data["surf_temps"]
# data["forces"]
# data["torques"]

# data["D1_TO_J2000"]

# TPM only on Dimorphos

In [101]:
thermo_params = ThermoParams(  # [Michel+2016; Naidu+2020]
    A_B   = 0.059,  # Bolometric  Bond albedo
    A_TH  = 0.0,
    k     = 0.125,
    ρ     = 2170.,
    Cp    = 600.,
    ϵ     = 0.9,
    t_bgn = et_range[begin],
    t_end = et_range[end],
    Nt    = length(et_range),
    z_max = 0.6,
    Nz    = 41,
    P     = SPICE.convrt(DIDYMOS[:P], "hours", "seconds"),
);

In [105]:
savepath = "tmp.jld2"
run_TPM!(shape2, et_range, sun_d2, thermo_params, savepath, save_range)

jldopen(savepath, "r+") do file
    file["D2_TO_J2000"] = D2_TO_J2000[save_range]
end;

In [113]:
draw(shape2; data=:temperature, colormap=:vik)

GLMakie.Screen(...)

# TPM on Binary Asteroid

In [153]:
thermo_params = ThermoParams(  # [Michel+2016; Naidu+2020]
    A_B   = 0.059,  # Bolometric  Bond albedo
    A_TH  = 0.0,
    k     = 0.125,
    ρ     = 2170.,
    Cp    = 600.,
    ϵ     = 0.9,
    t_bgn = et_range[begin],
    t_end = et_range[end],
    Nt    = length(et_range),
    z_max = 0.6,
    Nz    = 41,
    P     = SPICE.convrt(DIDYMOS[:P], "hours", "seconds"),
);

In [226]:
savepath = "tmp.jld2"
run_TPM!((shape1, shape2), et_range, (sun_d1, sun_d2), D1_TO_D2, thermo_params, savepath, save_range)

# jldopen(savepath, "r+") do file
#     file["D2_TO_J2000"] = D2_TO_J2000[save_range]
# end;

0.004908200959444313
0.006551827679886114
0.008329399176956262
0.010161444588644533
0.011970591161553563
0.013740552806529402
0.015443793935820255
0.016890170463791886
0.018212650663386756
0.019396890182927992
0.02049944259802724
0.02164042242706016
0.022760684372657092
0.0237300171053291
0.02460368405210114
0.025240140344111626
0.02577663009659019
0.02589219545918677
0.025879837708725657
0.025917663697898448
0.025946830464510365
0.026139398579365054
0.02640243279183433
0.026522649440375235
0.02671043284897507
0.026768892418698176
0.026830510722923556
0.02693801432851065
0.027099598723227737
0.027365773599381058
0.02751792342559084
0.027666592497869576
0.027955023137701773
0.02852282155509857
0.029192194981302863
0.029937870074893
0.03066226713665327
0.03132325661534503
0.03197056911030969
0.03252463206871922
0.03280173378675871
0.03305791802218202
0.03348502147062404
0.03413631534090421
0.035239678104880795
0.036586347775513624
0.03802676184082041
0.039511010911018184
0.04103941863177

In [193]:
# draw(shape1; data=:temperature, colormap=:vik)
# draw(shape2; data=:temperature, colormap=:vik)

GLMakie.Screen(...)

In [200]:
data = load("tmp.jld2");

In [225]:
# data["shapes"]
# data["et_range"]
# data["suns"]
# data["thermo_params"]
# data["surf_temps"][2]
# data["forces"]
# data["torques"]

# data["D1_TO_J2000"]

In [183]:
D1_TO_D2[1] * sun_d1[1] - d1_d2[1]

3-element Vector{Float64}:
 -1.4921259667528186e11
  5.883740903329407e10
  9.898824479821707e9

In [181]:
sun_d2[1]

3-element Vector{Float64}:
 -1.492125942952819e11
  5.883740903329409e10
  9.898824479821705e9

In [ ]:
function run_binary_TPM!(binary, thermo_params)
    @unpack shape1, shape2, orbit, mutual_orbit, spin1, spin2 = binary
    @unpack t_bgn, Δt, t_end, P = thermo_params

    init_temps_zero!(shape1, thermo_params)
    init_temps_zero!(shape2, thermo_params)

    ts = (t_bgn:Δt:t_end) * P
    df = DataFrame(
        t=Float64[], u=Float64[], ν=Float64[],
        f1_x=Float64[], f1_y=Float64[], f1_z=Float64[],
        τ1_x=Float64[], τ1_y=Float64[], τ1_z=Float64[],
        f2_x=Float64[], f2_y=Float64[], f2_z=Float64[],
        τ2_x=Float64[], τ2_y=Float64[], τ2_z=Float64[],
    )
    T₁ = Vector{Float64}[]
    T₂ = Vector{Float64}[]
    
    for t in ts
        update!(binary, t)
        
        r̂☉ = normalize(orbit.r) * -1  # Shift the origin from the sun to the body
        r̂☉₁ = orbit_to_body(r̂☉, spin1)
        r̂☉₂ = orbit_to_body(r̂☉, spin2)
        
        update_flux_sun!(shape1, orbit.F☉, r̂☉₁)
        update_flux_scat_single!(shape1, thermo_params)
        update_flux_rad_single!(shape1, thermo_params)
        
        update_flux_sun!(shape2, orbit.F☉, r̂☉₂)
        update_flux_scat_single!(shape2, thermo_params)
        update_flux_rad_single!(shape2, thermo_params)

        #### Mutual shadowing check ####
        ## Assuming the poles of rotation and mutual orbit coincide

        r̂☉ = rotateZ(r̂☉₁, -spin1.ϕ)  # Sun's direction in the mutable orbit frame
        # r̂☉ = rotateZ(r̂☉₂, -spin2.ϕ)

        r̂₁ = normalize(mutual_orbit.r₁)
        r̂₂ = normalize(mutual_orbit.r₂)
        sep1 = acos(r̂₁ ⋅ r̂☉)
        sep2 = acos(r̂₂ ⋅ r̂☉)
        
        if (t > 100 * spin2.P) && (rad2deg(min(sep1, sep2)) < 20)
            # println(t, ", ", rad2deg(sep1), ", ", rad2deg(sep2))

            for f₁ in shape1.facets
                ## Into the mutable orbit frame
                A₁ = rotateZ(f₁.A, -spin1.ϕ) + mutual_orbit.r₁
                B₁ = rotateZ(f₁.B, -spin1.ϕ) + mutual_orbit.r₁
                C₁ = rotateZ(f₁.C, -spin1.ϕ) + mutual_orbit.r₁
                center1 = rotateZ(f₁.center, -spin1.ϕ) + mutual_orbit.r₁
                # normal1 = rotateZ(f₁.normal, -spin1.ϕ)
                for f₂ in shape2.facets
                    ## Into the mutable orbit frame
                    A₂ = rotateZ(f₂.A, -spin2.ϕ) + mutual_orbit.r₂
                    B₂ = rotateZ(f₂.B, -spin2.ϕ) + mutual_orbit.r₂
                    C₂ = rotateZ(f₂.C, -spin2.ϕ) + mutual_orbit.r₂
                    center2 = rotateZ(f₂.center, -spin2.ϕ) + mutual_orbit.r₂
                    # normal2 = rotateZ(f₂.normal, -spin2.ϕ)

                    raycast(A₂ - center1, B₂ - center1, C₂ - center1, r̂☉) && (f₁.flux.sun = 0)
                    raycast(A₁ - center2, B₁ - center2, C₁ - center2, r̂☉) && (f₂.flux.sun = 0)
                end
            end
        end

        #### Mutual heating check ####
        ## Ignored for now

        update_force!(shape1, thermo_params)
        update_force!(shape2, thermo_params)
        sum_force_torque!(shape1)
        sum_force_torque!(shape2)

        f1 = body_to_orbit(SVector{3}(shape1.force),  spin1)  # Orbital plane frame
        τ1 = body_to_orbit(SVector{3}(shape1.torque), spin1)  # Orbital plane frame

        f2 = body_to_orbit(SVector{3}(shape2.force),  spin2)  # Orbital plane frame
        τ2 = body_to_orbit(SVector{3}(shape2.torque), spin2)  # Orbital plane frame
        
        update_temps!(shape1, thermo_params)
        update_temps!(shape2, thermo_params)

        push!(df, (t, orbit.u, orbit.ν, f1..., τ1..., f2..., τ2...))

        push!(T₁, surface_temperature(shape1))
        push!(T₂, surface_temperature(shape2))
    end
    df, T₁, T₂
end